In [1]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import sys
sys.path.append("/home/mylab-pharma/Code/tuele/XO/src/common")
from pharmacy_common import PharmacyCommon
#class to encode smiles
common = PharmacyCommon()
from tqdm import tqdm

Import screening data

In [4]:
all_screening_dataset_path = "../../data/screening_dataset/13. natural compounds.xlsx"
all_screening_dataset = pd.read_excel(all_screening_dataset_path)
print(len(all_screening_dataset))
all_screening_dataset.head(10)

407270


,SMILES,COCONUT id
0,O=C1OC2C(C(=C)C)CC1C3(O)CC4OC54C(=O)OC[CH]253C,CNP0000001
1,O=C(O)C=1C(=O)C(O)(CC(=O)C1N)C2OC(COC(=O)C)C(O...,CNP0000002
2,O=C1OC2C(O)C=CC3C4=C5C(=O)C=6C(OC)=CC=C(OC)C6C...,CNP0000003
3,O=C1OC2C(O)C=CC3C4=C5C(=O)C=6C=CC=C(O)C6C(=O)C...,CNP0000004
4,O=C(OC1CC(C)(CCOC(=O)C(C)(C)C)C2CC(C)(C)CC2C1O...,CNP0000005
5,O=C1OC2C(O)C=CC3C4=C5C(=O)C=6C=CC=C(O)C6C(=O)C...,CNP0000006
6,O=C1OC(C)C(O)C12C(O)C(O)CCC2C,CNP0000007
7,O=C1OC2C(OC(=O)C)C=CC3C4=C5C(=O)C=6C(OC(=O)C)=...,CNP0000008
8,O=C(OCCC1(C)CC(OC(=O)C)C(OC(=O)C)C2CC(C)(C)CC21)C,CNP0000009
9,O=CCC1(C)CC(=O)C(O)C2CC(C)(C)CC21,CNP0000010


Checking Duplicate SMILES

In [5]:
smiles_col_name = "SMILES"
cid_col_name = "COCONUT id"

In [7]:
screening_smiles = all_screening_dataset["SMILES"].unique()
len(screening_smiles)

407270

Checking if the SMILES are encodedable

In [8]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm

def check_encodable(data) -> np.ndarray:
    result = []
    error_data = []
    with tqdm(total=len(data), desc='Progress') as pbar:
        for i in data:
            try:
                mol = Chem.MolFromSmiles(i)
                fpts = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2,1024))
                result.append(i)
            except:
                error_data.append(i)
            pbar.update(1)  # Update the progress bar
    return result, error_data
encodable_smile, error_smile = check_encodable(screening_smiles)

Progress:   4%|▍         | 15930/407270 [00:05<02:11, 2984.19it/s][11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 11 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 7 N, 4, is greater than permitted
Progress:   4%|▍         | 16521/407270 [00:05<02:18, 2822.29it/s][11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 1 N, 4, is greater than permitted
[11:30:35] Explicit valence for atom # 17 N, 4, is greater than 

In [9]:
print(len(encodable_smile), len(error_smile)) 

406925 345


In [10]:
encoded_df = pd.DataFrame({"SMILES": encodable_smile})
# check_encode = common.gen_maccs_fpts(encoded_df["SMILES"])
check_encode = common.gen_ecfp4_fpts(encoded_df["SMILES"],1024)
encoded_df.head()

Progress:   0%|          | 0/406925 [00:00<?, ?it/s]

Progress: 100%|██████████| 406925/406925 [02:38<00:00, 2565.46it/s]


,SMILES
0,O=C(O)C=1C(=O)C(O)(CC(=O)C1N)C2OC(COC(=O)C)C(O...
1,O=C1OC2C(O)C=CC3C4=C5C(=O)C=6C(OC)=CC=C(OC)C6C...
2,O=C1OC2C(O)C=CC3C4=C5C(=O)C=6C=CC=C(O)C6C(=O)C...
3,O=C(OC1CC(C)(CCOC(=O)C(C)(C)C)C2CC(C)(C)CC2C1O...
4,O=C1OC2C(O)C=CC3C4=C5C(=O)C=6C=CC=C(O)C6C(=O)C...


In [11]:
print(len(encoded_df))

406925


In [12]:
error_data = pd.DataFrame({"SMILES":error_smile})
error_data.head(10)

,SMILES
0,O=C1OC2C(C(=C)C)CC1C3(O)CC4OC54C(=O)OC[CH]253C
1,OB1OB2(O)OB(O)OB(O)(O1)O2
2,C#NC1(C)CCC(C(=C)CCC=C(C)C)CC1
3,C#NC12C=C(CCC2(C)CCCC1C)C(C)C
4,C#NC12CC=C3CC(C)(C)CC4CC(C)C(CCC1C)C2C34
5,C#NC1C2(C)CCC3C(C2)C(CC13C)C(C)C
6,C#NC12CCC(C3=C(C=C(C)C)CC(C)C(CCC1C)C32)C
7,C#NC12CCC(=C)C3C(C=C(C)C)CC(C)C(CCC1C)C32
8,C=1C=CC2=C(C1)NC3=C2C=CN(=C3C)C
9,C=1C=CC=2C(C=CN(C)(C)C)=CNC2C1


Write to file

In [17]:
with pd.ExcelWriter("/home/mylab-pharma/Code/tuele/XO/data/screening_dataset/preprocessed_screening_dataset.xlsx", engine='openpyxl') as writer:
    all_screening_dataset.to_excel(writer, sheet_name='original_data', index=False)
    encoded_df.to_excel(writer, sheet_name="final_screening_data", index=False)
    error_data.to_excel(writer,sheet_name="unencoded_smiles",index=False)